In [ ]:
# From http://stackoverflow.com/questions/8367609/most-elegant-approach-for-writing-json-data-to-a-relational-database-using-djang

def create_or_update_and_get(model_class, data):
    get_or_create_kwargs = {
        model_class._meta.pk.name: data.pop(model_class._meta.pk.name)
    }
    try:
        # get
        instance = model_class.objects.get(**get_or_create_kwargs)
    except model_class.DoesNotExist:
        # create
        instance = model_class(**get_or_create_kwargs)
    # update (or finish creating)
    for key,value in data.items():
        field = model_class._meta.get_field(key)
        if not field:
            continue
        if isinstance(field, models.ManyToManyField):
            # can't add m2m until parent is saved
            continue
        elif isinstance(field, models.ForeignKey) and hasattr(value, 'items'):
            rel_instance = create_or_update_and_get(field.rel.to, value)
            setattr(instance, key, rel_instance)
        else:
            setattr(instance, key, value)
    instance.save()
    # now add the m2m relations
    for field in model_class._meta.many_to_many:
        if field.name in data and hasattr(data[field.name], 'append'):
            for obj in data[field.name]:
                rel_instance = create_or_update_and_get(field.rel.to, obj)
                getattr(instance, field.name).add(rel_instance)
    return instance

# for example:
#from django.utils.simplejson import simplejson as json

#data = json.loads(ninja_json)
#ninja = create_or_update_and_get(NinjaData, data)

In [ ]:
!ls publications

In [ ]:
filename = 'publications/extracted_taxon_names_Nx3SUM1.json'
with open(filename) as json_data:
    d = json.load(json_data)

In [ ]:
data = d['resolved_names'][1]     
data    

In [ ]:
for name in d['resolved_names']:
    print('supplied_name_string: {}   is_known_name: {}'.format(
          name['supplied_name_string'], 
          name['is_known_name']))
    results = name['results']
    print(results)
    print()

In [ ]:
field_names = []
for field in NameFinderResults._meta.get_fields():
    if field.name not in ['id', 'pub']:
        field_names.append(field.name)
#field_names

In [4]:
from publications.models import Publication
from taxonomy.models import NameFinderResult
from publications.models import CustomFile
import json

def json_to_name_finder_results(pub, name_finder_json):
    d = name_finder_json
    for name in d['resolved_names']:
        x = NameFinderResult()
        x.pub = pub
        x.is_known_name = name['is_known_name']
        x.supplied_name_string = name['supplied_name_string']

        print('supplied_name_string: {}   is_known_name: {}'.format(
              name['supplied_name_string'], 
              name['is_known_name']))
        results = name['results']
        if len(results) == 1:
            r = results[0]
            x.classification_path_ranks = r.get('classification_path_ranks','')
            x.classification_path = r.get('classification_path','')
            x.current_name_string = r.get('current_name_string','')
            x.imported_at = r.get('imported_at','')
            x.canonical_form = r.get('canonical_form','')
            x.data_source_id = r.get('data_source_id','')
            x.match_value = r.get('match_value','')
            x.data_source_title = r.get('data_source_title','')
            x.gni_uuid = r.get('gni_uuid','')
            x.edit_distance = r.get('edit_distance','')
            x.match_type = r.get('match_type','')
            x.name_string = r.get('name_string','')
            x.current_taxon_id = r.get('current_taxon_id','')
            x.taxon_id = r.get('taxon_id','')
            x.prescore = r.get('prescore','')
            x.classification_path_ids = r.get('classification_path_ids','')
            x.score = r.get('score','')
        x.save()

supplied_name_string: Mantodea   is_known_name: True
supplied_name_string: Istor   is_known_name: True
supplied_name_string: Tenodera australasiae   is_known_name: True
supplied_name_string: Orthodera burmeisteri   is_known_name: True
supplied_name_string: Statilia pallida   is_known_name: True
supplied_name_string: Tenodera aridifolia sinensis   is_known_name: False
supplied_name_string: Acromantis   is_known_name: True
supplied_name_string: Anga ur   is_known_name: False
supplied_name_string: Acromantis palauana   is_known_name: True


In [ ]:
x.